In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [14]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, out_channels))
        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        x = self.lins[0](x)
        return torch.sigmoid(x)

In [15]:
device=0
log_steps=1
num_layers=2
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [16]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [17]:
adj_t = data.adj_t.to(device)

In [18]:
torch.manual_seed(12345)

In [19]:
model = SAGE(hidden_channels, hidden_channels,
             hidden_channels, num_layers,
             dropout).to(device)

In [20]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [21]:
predictor = LinkPredictor(hidden_channels, hidden_channels, 1,
                          num_layers, dropout).to(device)

In [22]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [23]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [24]:
model.reset_parameters()
predictor.reset_parameters()

In [25]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [26]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [27]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 0.8601
Epoch: 04, Loss: 0.6391
Epoch: 06, Loss: 0.5524
Epoch: 08, Loss: 0.4266
Epoch: 10, Loss: 0.3719
Epoch: 12, Loss: 0.3345
Epoch: 14, Loss: 0.3033
Epoch: 16, Loss: 0.2865
Epoch: 18, Loss: 0.2588
Epoch: 20, Loss: 0.2445
Epoch: 22, Loss: 0.2291
Epoch: 24, Loss: 0.2134
Epoch: 26, Loss: 0.1993
Epoch: 28, Loss: 0.1862
Epoch: 30, Loss: 0.1741
Epoch: 32, Loss: 0.1615
Epoch: 34, Loss: 0.1520
Epoch: 36, Loss: 0.1444
Epoch: 38, Loss: 0.1374
Epoch: 40, Loss: 0.1315
Epoch: 42, Loss: 0.1287
Epoch: 44, Loss: 0.1231
Epoch: 46, Loss: 0.1181
Epoch: 48, Loss: 0.1152
Epoch: 50, Loss: 0.1110
Epoch: 52, Loss: 0.1076
Epoch: 54, Loss: 0.1040
Epoch: 56, Loss: 0.0999
Epoch: 58, Loss: 0.0984
Epoch: 60, Loss: 0.0973
Epoch: 62, Loss: 0.0941
Epoch: 64, Loss: 0.0920
Epoch: 66, Loss: 0.0895
Epoch: 68, Loss: 0.0879
Epoch: 70, Loss: 0.0866
Epoch: 72, Loss: 0.0857
Epoch: 74, Loss: 0.0840
Epoch: 76, Loss: 0.0827
Epoch: 78, Loss: 0.0806
Epoch: 80, Loss: 0.0800
Epoch: 82, Loss: 0.0785
Epoch: 84, Loss:

In [28]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
    (1): SAGEConv(50, 50)
  )
)

In [29]:
emb.weight

Parameter containing:
tensor([[-0.1737,  0.1814,  0.0045,  ..., -0.2158,  0.3789, -0.2738],
        [-0.1065,  0.3134, -0.1303,  ...,  0.1988,  0.0382,  0.1104],
        [-0.0418,  0.0627,  0.0016,  ...,  0.1750, -0.1142, -0.0866],
        ...,
        [ 0.1690,  0.0453, -0.1152,  ..., -0.0999,  0.5872,  0.0765],
        [ 0.1364, -0.1245,  0.3883,  ...,  0.1917,  0.2049,  0.1302],
        [ 0.1147,  0.0655, -0.1746,  ...,  0.0571, -0.1090,  0.0571]],
       device='cuda:0', requires_grad=True)

In [30]:
h = model(emb.weight, adj_t)

In [31]:
h

tensor([[ 2.8511,  0.9312, -1.0075,  ..., -2.7549, -1.0603,  0.4561],
        [-3.6245, -1.5313,  0.2473,  ..., -0.2251,  0.7152,  3.3667],
        [-1.0060, -0.8207,  0.1550,  ..., -1.4256,  0.7128,  2.0857],
        ...,
        [-1.4319, -0.2280, -2.7242,  ..., -1.3183, -0.1505,  0.9760],
        [ 2.3265, -0.6230,  1.3398,  ..., -0.7416, -2.9309,  1.9714],
        [-1.1666, -3.2060,  4.8194,  ...,  0.8934,  2.6041,  4.6815]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [42]:
for param in predictor.parameters():
    print(param)

Parameter containing:
tensor([[ 0.2690, -0.2627,  0.2131, -0.2238,  0.2750,  0.2987, -0.2024, -0.2181,
          0.2002, -0.1801,  0.2764, -0.1878,  0.2734, -0.1717,  0.2402, -0.1544,
         -0.1996, -0.2068, -0.2155, -0.2005, -0.2252, -0.1595,  0.3166, -0.2234,
          0.2373, -0.1778,  0.1989, -0.2409, -0.1333, -0.1749,  0.2626, -0.2019,
         -0.2545, -0.2634,  0.2787,  0.2762,  0.2384,  0.2832, -0.1789,  0.3228,
         -0.2117, -0.1518, -0.1689, -0.1990, -0.2208, -0.1246, -0.1446, -0.3996,
          0.2594, -0.2188]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.6925], device='cuda:0', requires_grad=True)


In [32]:
predictor(h[0], h[411])

tensor([0.9949], device='cuda:0', grad_fn=<SigmoidBackward>)

In [33]:
predictor(h[0], h[2])

tensor([1.2293e-08], device='cuda:0', grad_fn=<SigmoidBackward>)

In [34]:
(h[0] * h[2]).sum()

tensor(40.6949, device='cuda:0', grad_fn=<SumBackward0>)

In [35]:
(h[0] * h[411]).sum()

tensor(162.8968, device='cuda:0', grad_fn=<SumBackward0>)

In [37]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_18_h.npy', h.cpu().detach().numpy())

In [38]:
torch.save(model, '../../models/graphsage_link_pred/18_model.pt')

In [39]:
torch.save(emb, '../../models/graphsage_link_pred/18_embedding.pt')

In [40]:
torch.save(predictor, '../../models/graphsage_link_pred/18_predictor.pt')